In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from datetime import datetime, timedelta, timezone
import json 
import os
import base64
import requests
print("DONE")

DONE


In [2]:
#constant
urlzalo = 'https://id.zalo.me/account?continue=https%3A%2F%2Fchat.zalo.me%2F'
# Lưu ảnh
save_img = 'D:/Learn/Zalo_crawls/img'
save_video = 'D:/Learn/Zalo_crawls/video'
save_file = 'D:/Learn/Zalo_crawls/file'
with open("data.json", "r", encoding="utf-8") as file:
    data = json.load(file)        
with open("information.json", "r", encoding="utf-8") as file:
    information = json.load(file)
with open("group.json", "r", encoding="utf-8") as file:
    group = json.load(file)

last_save_time = 0      
#tool 
#xpath = "https://chromewebstore.google.com/detail/xpath-helper-20/nlfdopnkeodpiondihlgnhgpjieoopbj?utm_source=ext_app_menu"
print("DONE")

DONE


In [7]:
data = []
information = []
group = []
save(data,information,group)


In [11]:
# đây là hàm khởi taoj ban đầuđầu
def start():

    # chrome_options = Options()
    # chrome_options.add_experimental_option("prefs", {
    #     "download.default_directory": save_file,  # Đổi thư mục tải xuống
    #     "download.prompt_for_download": False,  # Không hiển thị hộp thoại yêu cầu tải xuống
    #     "directory_upgrade": True
    # })
    driver = webdriver.Chrome()
    driver.get(urlzalo)
    return driver
# Lưu vào file
def save(data,information,group):
    with open("data.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4) 
    with open("information.json", "w", encoding="utf-8") as file:
        json.dump(information, file, ensure_ascii=False, indent=4) 
    with open("group.json", "w", encoding="utf-8") as file:
        json.dump(group, file, ensure_ascii=False, indent=4) 
# đây là hàm login bằng password và sddt 
def login(driver):
    zdropdown= driver.find_element(By.XPATH,"//div[@class='zdropdown']/button")
    zdropdown.click()
    time.sleep(1)
    zdropdown_container= driver.find_element(By.XPATH,"//div[@class='zdropdown-container show']")
    zdropdown_container.click() 
    time.sleep(1)
    phone = driver.find_element(By.XPATH,"//input[@id='input-phone']")
    phone.send_keys('0985135941')
    time.sleep(1)
    password = driver.find_element(By.XPATH,"//input[@type='password']")
    password.send_keys('Lkcntt2mtak55')
    time.sleep(1)
    login= driver.find_element(By.XPATH,"//a[@class='btn btn--m block first ']")
    login.click() 

# Hàm này là để chuyển từ chưa đọc qua tất cả sẽ reload lại trang 
def reload(driver):
    # all
    all = driver.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_ALL']")
    all.click()
    time.sleep(0.5)
    pinner = driver.find_element(By.XPATH,"//div-16")
    pinner.click()
    time.sleep(0.5)
#unread
    unread = driver.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_UNREAD_LABEL']")
    unread.click() 
#đây là ham kiểm tra xẻm có tin nhăn mới không 
def check_new_message(driver):
    try:
        mes_gr = driver.find_element(By.XPATH,"//div[@class='msg-item ']")
        return mes_gr
    except:
        return None



def predict_time_from_id(id_number):
    # Chênh lệch quan sát được (~ -25 giây) từ các ví dụ trước
    time_offset = 25000  # 25 giây
    
    # Thêm chênh lệch vào ID (số thứ hai)
    predicted_millis = int(id_number) + time_offset
    
    # Chuyển đổi mili giây thành thời gian thực với múi giờ UTC
    predicted_time = datetime.fromtimestamp(predicted_millis / 1000, timezone.utc)
    
    # Điều chỉnh theo múi giờ (Ví dụ múi giờ GMT+7)
    predicted_time += timedelta(hours=7)
    
    # Trả về thời gian dự đoán đã điều chỉnh
    return predicted_time.strftime('%Y-%m-%d %H:%M')


# cuộn trang xuống 
def scroll(driver):
    bb_mes = driver.find_element(By.XPATH,"//div[@id='messageViewContainer']/div")
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop - 1000;", bb_mes)
def download_img(driver, img_url,user,group):
    if img_url.startswith("blob:"): 
        # Tạo URL blob thành dữ liệu base64
        img_base64 = driver.execute_script("""
            return fetch(arguments[0])
                .then(res => res.blob())
                .then(blob => new Promise((resolve, reject) => {
                    const reader = new FileReader();
                    reader.onloadend = () => resolve(reader.result);
                    reader.onerror = reject;
                    reader.readAsDataURL(blob);
                }));
        """, img_url)
        # Giải mã dữ liệu từ base64
        header, encoded = img_base64.split(",", 1)
        file_extension = header.split(";")[0].split("/")[-1]
        image_data = base64.b64decode(encoded)
        # Trích xuất tên file từ blob URL
        file_name = img_url.split(":")[-1].split("/")[-1]  # Lấy '8733691f-bc6d-4bfa-995f-a099901a7dad'

        # Đường dẫn thư mục lưu ảnh
        group_dir = os.path.join(save_img, group)
        user_dir = os.path.join(group_dir, user)
        
        # Tạo thư mục nếu chưa tồn tại
        os.makedirs(user_dir, exist_ok=True)

        # Lưu ảnh vào thư mục con
        save_path = os.path.join(user_dir, f"{file_name}.{file_extension}")
        
        with open(save_path, "wb") as f:
            f.write(image_data)
        return f"{file_name}.{file_extension}"
    else:
        return "URL không hợp lệ hoặc không phải blob URL."

def download_video(video_url,user,group):
    # Tách `id` từ URL video (blob URL hoặc URL thông thường)
    video_id = video_url.split(":")[-1].split("/")[-1]  # Lấy '8733691f-bc6d-4bfa-995f-a099901a7dad'
    file_name = f"{video_id}.mp4"
    # Tạo thư mục lưu video nếu chưa tồn tại
    
    group_dir = os.path.join(save_video, group)
    user_dir = os.path.join(group_dir, user)
    os.makedirs(user_dir, exist_ok=True)
       
    # Đường dẫn lưu video
    save_path = os.path.join(user_dir, file_name)

    # Tải video về
    response = requests.get(video_url, stream=True)
    if response.status_code == 200:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Video đã được tải về thành công: {save_path}")
    else:
        print("Không thể tải video. Mã lỗi:", response.status_code)

    
      
def update_data(driver,list_bb_mes,id_group,max_time,first_user):
    information = []
    data =[]
    file_data = []
    type = None
    i = 0 
    print("thu thập được",len(list_bb_mes))

    for bb_mes in reversed(list_bb_mes): # đoạn này xoay ngược lại cái list để thu thập từ dưới lên
        # đoạn này thì lấy thông tin nè
        bb_id = bb_mes.find_element(By.XPATH,".//div[@class='message-action']/div") # cái thẻ div có thuộc tính data-id 
        data_id = bb_id.get_attribute("data-id") # nó kiểu này image-mCntr_6237497978845-1737099761663-3408274176834806327-g1985432875231041526
        # cái số đầu tiên là id , 2 là thời gian, 3 là id user , 4 là id group
        # cái này chỉ có mỗi tin nhắn và hình ảnh có thôi
        

        # đoạn này là lấy time của box tin nhắn 
        full_time = bb_mes.get_attribute("id") # sẽ có dạng bb_msg_id_1737099761663
        time_id = int(full_time.split("_")[-1]) # ta lấy mỗi cái id cuối thôi

        #giờ so sánh nếu cái time của tin nhắn hiện tại mà bé hơn hoặc bangừ cái max_time 
        # thì chứng tỏ là đây là tin nhắn đã crawl rồi
        if time_id <= max_time: 
            print(i,"dừng lại giữa chừng vì",time_id ,"<=",max_time,)
            if file_data:
                print('file_data 1 ',file_data)
                if not type:
                    type = first_user
                for file in file_data:     
                    file['user'] = type
                    data.append(file)
            return data, information
        
        i+=1
        
        ### message 
        if data_id and "ReceivedMsg_Text" in data_id:
            # Tách chuỗi bb_id_id để lấy các thành phần
            bb_id_id= bb_id.get_attribute("id")
            bb_id_text = bb_id.text
            parts = bb_id_id.split("-")
            type= parts[3]
            data.append({
                'id': parts[1],
                'type': 'message',
                'content': bb_id_text,
                'time': time_id,
                'user': parts[3],
                'group': id_group
            })
        ### photos
        elif data_id and "ReceivedMsg_Photo" in data_id:
 
            src = bb_id.find_element(By.XPATH,".//img").get_attribute("src")
            bb_id_id= bb_id.get_attribute("id")
            parts = bb_id_id.split("-")
            type= parts[3]
            try:
                text = bb_id.find_element(By.XPATH,".//span").text
            except:
                text = None
            data.append({
                'id': parts[1],
                'type': 'photo',
                'content': download_img(driver, src,parts[3],id_group),
                'text': text,
                'time': time_id,
                'user': parts[3],
                'group': id_group
            })
        ### info
        elif data_id and "DisabledTargetEventLayer" in data_id:
            info = bb_id.find_element(By.XPATH,".//div[@class='contact-card__info-container']").text 
            data.append({
                'type': 'info',
                'content': info,
                'time': time_id,
                'group':id_group
            })
        else:
            class_name = bb_id.get_attribute("class")
            ##file
            if 'file' in class_name:
        
                name = bb_id.find_element(By.XPATH,".//div-b14").get_attribute("title")
                size = bb_id.find_element(By.XPATH,".//span[@class='file-message__content-info-size']").text
                bb_mes.find_element(By.XPATH,".//a").click()
                file_data.append({
                    'type': 'file',
                    'content': name,
                    'size': size,
                    'time': time_id,
                    'user': None,
                    'group':id_group
                })
            ###video
            elif 'video' in class_name:
                
                show_video = bb_mes.find_element(By.XPATH,".//div[@class='video-message__background-overlay video-message__background-overlay--clickable rounded-t-5 rounded-b-5']")
                show_video.click()
                video_url = None
                while not video_url:
                    try:
                        time.sleep(0.5)
                        video_url= driver.find_element(By.XPATH,"//video").get_attribute("src")
                    except:
                        pass
                user_name = bb_mes.find_element(By.XPATH,".//img").get_attribute("crossorigin")# lấy ra user name của video từ thuộc tính crossorigin của thẻ img
                if not type:
                    type = user_name
                download_video(video_url,user_name,id_group)
                driver.find_element(By.XPATH,"//i[@class='btn dark btn--m  fa fa-close']").click() # nhấn close để thoát khỏi trình show video
                data.append({
                    'type': 'video',
                    'content': video_url,
                    'time': time_id,
                    'user': user_name,
                    'group':id_group
                })
            elif 'group-photo' in class_name:
                a = bb_mes.find_elements(By.XPATH,".//div[@class='local-overlaying-message-status-wrapper']/div")
                for bb_id in a: 
                    bb_class = bb_id.get_attribute('class')
                    if "group-photo" in bb_class:
                        bb_id.click()
                        video_url = None
                        while not video_url:
                            try:
                                time.sleep(0.5)
                                video_url= driver.find_element(By.XPATH,"//video").get_attribute("src")
                            except:
                                pass
                        bb_id_id= bb_id.get_attribute("id")
                        parts = bb_id_id.split("-")
                        type= parts[3]
                        id_group = parts[-1]
                        download_video(video_url,parts[3],id_group)
                        driver.find_element(By.XPATH,"//i[@class='btn dark btn--m  fa fa-close']").click() # nhấn close để thoát khỏi trình show video

                        data.append({
                            'id': parts[1],
                            'type': 'video_in_album',
                            'content': video_url,
                            'time': time_id,
                            'user': parts[3],
                            'group':id_group
                        })
                    elif "picture" in bb_class:                                                                                                       
                        src = bb_id.find_element(By.XPATH,".//img").get_attribute("src")
                        bb_id_id= bb_id.find_element(By.XPATH,".//div").get_attribute("id")
                        
                        parts = bb_id_id.split("-")
                        type= parts[3]
                        id_group = parts[-1]
                        data.append({
                            'id': parts[1],
                            'type': 'photo_in_album',
                            'content': download_img(driver, src,parts[3],id_group),
                            'time': time_id,
                            'user': parts[3],
                            'group': id_group
                        })
        name_user = None
        # lấy name nguòi dùng nếu có
        try:
            name_user = bb_mes.find_element(By.XPATH,".//div[@class='truncate']").text
            print('data name_user',name_user)
            # Tách chuỗi bb_id_id để lấy các thành phần
            bb_id_id= bb_id.get_attribute("id")
            print('data bb_id_id',bb_id_id)
            parts = bb_id_id.split("-")

            information.append({
                'id': parts[3],
                'name':name_user
            })
            if file_data:
                if not type: 
                    print('no type')
                    type = name_user
    
                for file in file_data:     
                    file['user'] = type
                    data.append(file)
                file_data =[]
                type = None

        except:
            pass

            # else:
            #     data.append({
            #         'type': 'mp3',
            #         'content': bb_id.text
            #     })
        # Sử dụng set để loại bỏ các phần tử trùng lặp
    if file_data:
        print('file_data 2 ',file_data)
        if not type:
            type = first_user
        for file in file_data:     
            file['user'] = type
            data.append(file)
    
    return data, information

def update_information(information,new_info):
        ## Xosd trung lặp trong danh sách mới
        # Sử dụng set để loại bỏ các phần tử trùng lặp
        unique_data = {entry['id']: entry for entry in new_info}.values()
        
        # Chuyển lại thành danh sách
        new_info = list(unique_data)
        
        # Chèn vào danh sach cũ , nếu mà trungf id mà có name thì thay thế còn không name thì giữ nguyên, thêm mới cái ko có
        # Chuyển danh sách gốc thành dictionary để dễ thao tác
        original_dict = {entry['id']: entry for entry in information}
        
        # Duyệt qua danh sách mới và xử lý
        for new_entry in new_info:
            id = new_entry['id']
            if id in original_dict:
                # Nếu trùng id, kiểm tra name
                if new_entry['name'] is not None:
                    current_name = original_dict[id]['name']
                    if current_name != new_entry['name']:  # Chỉ lưu nếu tên thay đổi
                        original_dict[id]['history'].append(current_name)
                    # Cập nhật name
                    original_dict[id]['name'] = new_entry['name']
            else:
                    # Nếu không trùng, thêm phần tử mới
                if 'history' not in new_entry:
                    new_entry['history'] = []
                original_dict[id] = new_entry

        # Chuyển lại thành danh sách
        return list(original_dict.values())

def update_group(group,max_time,id_group,name_gr,first_user):
    
    # cap nhat group 
    found = False
    for item in group:
        if id_group and item['id'] == id_group:
            # Nếu đã tồn tại, cập nhật max_time
            if item['name'] != name_gr:  # Kiểm tra nếu tên thay đổi
                # if 'history' not in item:  # Đảm bảo trường history tồn tại
                #     item['history'] = []
                item['history'].append(item['name'])  # Lưu tên cũ vào history
            item['name'] = name_gr  # Cập nhật tên mới
            item['max_time'] = max_time  # Cập nhật max_time
            item['first_user'] = first_user  # Cập nhật max_time
            found = True
            break
        elif item['name'] == name_gr:
            if not item.get('id') and id_group:  # Trường hợp cần cập nhật id_group
                item['id'] = id_group
            item['max_time'] = max_time  # Cập nhật max_time
            item['first_user'] = first_user  # Cập nhật max_time
            found = True
            break             
    if not found:
        # Nếu không tìm thấy phần tử, thêm mới
        group.append({
            'id': id_group,
            'name': name_gr,
            'max_time': max_time,
            'first_user': first_user,
             'history': []  # Thêm max_id mới vào
        })
    return group
def count_by_type(data):
    type_counts = {}
    for item in data:
        type_value = item.get('type')
        if type_value:
            type_counts[type_value] = type_counts.get(type_value, 0) + 1
    return type_counts
def run(driver,driver2):   
    with open("data.json", "r", encoding="utf-8") as file:
        data = json.load(file)        
    with open("information.json", "r", encoding="utf-8") as file:
        information = json.load(file)
    with open("group.json", "r", encoding="utf-8") as file:
        group = json.load(file)
    while True:
        try:
            stop = driver2.find_elements(By.XPATH,"//div[@data-component='bubble-message']")[-1]
            if 'stop' in stop.text:
                mes_box = driver2.find_element(By.XPATH,"//div[@id='richInput']")
                mes_box.send_keys(f"Bot: OK") 
                time.sleep(0.5)
                mes_enter = driver2.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
                mes_enter.click()
                print("STOP")
                break

        except:
            pass
        #unread
    
        unread = driver.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_UNREAD_LABEL']")
        unread.click() 
        time.sleep(0.5)
        mes_gr= check_new_message(driver)
        if mes_gr:
            mes_gr.click()
            time.sleep(0.5)
            name_gr = driver.find_element(By.XPATH,"//div-b18").text
            id_group = None
            while not id_group:
                try:
                    id_group = driver.find_element(
                        By.XPATH,
                        "//div[@data-id='div_ReceivedMsg_Photo' or @data-id='div_LastReceivedMsg_Photo' or @data-id='div_ReceivedMsg_Text']"
                    ).get_attribute('id').split("-")[-1]

                    # group.append({
                    #         'id': id_group,
                    #         'name':name_gr,
                    #         'max_id': None
                    #     })

                except:
                    try:
                        overscroll = driver.find_element(By.XPATH,"//div[@class='flx-centralized']")
                        break 
                    except:
                        pass
                    scroll(driver)
            #đoạn này là lấy max_time ra để chạychạy
            # max_time là thời gian của tin nhắn cuối cùng crawl được từ nhóm 
            if id_group:
                max_time = next((item['max_time'] for item in group if item['id'] == id_group), None) # có gr avaf id rồirồi
                first_user = next((item['first_user'] for item in group if item['id'] == id_group), None) # có gr avaf id rồirồi
            else:
                max_time = next((item['max_time'] for item in group if item['name'] == name_gr), None) # có gr nhưng chưa có idid
                first_user = next((item['first_user'] for item in group if item['name'] == name_gr), None) # có gr nhưng chưa có idid

            if max_time:
                max_time = int(max_time) 
            else:
                max_time = 0 # chưa cso gr 
        
            list_bb_mes = driver.find_elements(By.XPATH,"//div[@data-component='bubble-message']")
            first_time = int(list_bb_mes[0].get_attribute("id").split("_")[-1])
            while first_time > max_time:
                scroll(driver)
                time.sleep(0.5)
                list_bb_mes = driver.find_elements(By.XPATH,"//div[@data-component='bubble-message']")
                first_time = int(list_bb_mes[0].get_attribute("id").split("_")[-1])
                time.sleep(0.5)
                try:
                    overscroll = driver.find_element(By.XPATH,"//div[@class='flx-centralized']")
                    break 
                except:
                    pass
            print(first_time,' run first_time <= max_time',max_time)
            new_max_time = int(list_bb_mes[-1].get_attribute("id").split("_")[-1]) # đoạn này lấy thời gian của tin nhắn cuối cùng
            print('run new_max_time', new_max_time)
            if not id_group:
                save_id_group = 'nameGroup_'+ name_gr
            else:
                save_id_group = id_group
            print(len(list_bb_mes))
            new_data, new_info= update_data(driver,list_bb_mes,save_id_group,max_time,first_user) # đoạn này crawwl
            print(' run new_data', new_data)
            print('run new_info', new_info)

            first_user = next((item['user'] for item in new_data if 'user' in item), None) # đây là id cuôi cùng của người dùng trong nhóm

            print('run first_user',first_user)
            group = update_group(group,new_max_time,id_group,name_gr,first_user)
            information = update_information(information,new_info)


            # đoạn này là nhảy ra cloud để tiến hành cập nhật lại data để tránh có tin nhắn mới vào làm ảnh hưởng
            all = driver.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_ALL']")
            all.click()
            pinner = driver.find_element(By.XPATH,"//div-16")
            pinner.click()

            #cập nhật lạ data 
            data.extend(new_data)
            save(data,information,group)
            time.sleep(5)
                # Kiểm tra xem đã đến 1 giờ kể từ lần lưu cuối chưa
            # current_time = time.time()
            # if current_time - last_save_time >= 3600:
            #  # 3600 giây = 1 giờ
            #     save(data, max_time)
            #     last_save_time = current_time 
            #     print("luu ",current_time)
            mes_box = driver2.find_element(By.XPATH,"//div[@id='richInput']")
            mes_box.send_keys(f"Bot: đã thu thập được {count_by_type(new_data)} và {len(new_info)} user mới trong group {name_gr}") 
            time.sleep(0.5)
            mes_enter = driver2.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
            mes_enter.click()
        else:  
            time.sleep(10)
            print('reload')
            reload(driver)
print("DONE")

DONE


In [4]:
driver= start()
driver2= start()
print("DONE")

DONE


In [12]:
flag_start = None
while True:
    try:
        flag_start = driver2.find_elements(By.XPATH,"//div[@data-component='bubble-message']")[-1]
        if 'start' in flag_start.text:
            run(driver,driver2)
            flag_start = None
            all = driver2.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_ALL']")
            all.click()
            pinner = driver2.find_element(By.XPATH,"//div-16")
            pinner.click()
        if 'done' in flag_start.text:
            mes_box = driver2.find_element(By.XPATH,"//div[@id='richInput']")
            mes_box.send_keys(f"Bot: đã dừng toàn bộ code") 
            time.sleep(0.5)
            mes_enter = driver2.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
            mes_enter.click()
            break
    except:
        pass 
    time.sleep(1)

reload
1737623099728  run first_time <= max_time 0
run new_max_time 1737623099728
1
thu thập được 1
data name_user Anonymous
data bb_id_id text-mCntr_6256752161882-1737623099728-3408274176834806327-g8294438841570765152
 run new_data [{'id': 'mCntr_6256752161882', 'type': 'message', 'content': 'Ja', 'time': 1737623099728, 'user': '3408274176834806327', 'group': 'g8294438841570765152'}]
run new_info [{'id': '3408274176834806327', 'name': 'Anonymous'}]
run first_user 3408274176834806327
reload
1737623099728  run first_time <= max_time 1737623099728
run new_max_time 1737623122244
2
thu thập được 2
data name_user did
data bb_id_id 
1 dừng lại giữa chừng vì 1737623099728 <= 1737623099728
file_data 1  [{'type': 'file', 'content': 'did', 'size': '122 B', 'time': 1737623122244, 'user': None, 'group': 'g8294438841570765152'}]
 run new_data [{'type': 'file', 'content': 'did', 'size': '122 B', 'time': 1737623122244, 'user': '3408274176834806327', 'group': 'g8294438841570765152'}]
run new_info []
r

In [43]:
driver2.find_elements(By.XPATH,"//div[@data-component='bubble-message']")[-1]
mes_box = driver2.find_element(By.XPATH,"//div[@id='richInput']")
mes_box.send_keys(f"Bot: đã dừng toàn bộ code") 
time.sleep(0.5)
mes_enter = driver2.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
mes_enter.click()

In [130]:
login(driver)
predict_time_from_id(1736761806386)

In [5]:
overscroll = driver.find_element(By.XPATH,"//div[@class='flx-centralized']")

In [109]:
max_time = 0
last_save_time = 9999999
data = []

In [4]:
a =[]
if a:
    print(1)
else:
    print(2)

2


In [6]:
driver.find_element(By.XPATH,"//div-b18").text

'DATA3'

reloadreload
reloadreload
có tin nhắn mớimới


ValueError: not enough values to unpack (expected 2, got 0)

In [5]:
list_bb_mes = driver.find_elements(By.XPATH,"//div[@data-component='bubble-message']")
len(list_bb_mes)

1

In [6]:
bb_mes = list_bb_mes[0]
bb_id = bb_mes.find_element(By.XPATH,".//div[@class='message-action']/div") # cái thẻ div có thuộc tính data-id 
data_id = bb_id.get_attribute("data-id") # nó kiểu này image-mCntr_6237497978845-1737099761663-3408274176834806327-g1985432875231041526
class_name = bb_id.get_attribute("class")
if 'group-photo' in class_name:
    a = bb_mes.find_elements(By.XPATH,".//div[@class='local-overlaying-message-status-wrapper']/div")
    for bb_id in a: 
        video ='card--group-photo__row__item__content'
        image = 'chat-message-picture__photooverlay no-shadow-border'
        bb_class = bb_id.get_attribute('class')
        if "group-photo" in bb_class:
            print('video')
            bb_id.click()
        elif "picture" in bb_class:
            print('photo')                                                                                                     


photo
video
photo
photo
video


ElementClickInterceptedException: Message: element click intercepted: Element <div id="video-mCntr_6240342660638-1737179261389-3408274176834806327-g3824825119537586287" class="card--group-photo__row__item__content" style="width: 88px; height: 158px; border-radius: 0px;">...</div> is not clickable at point (610, 428). Other element would receive the click: <video class="image-show__video" src="https://f123-zvc.dlmd.me/gr/dec5432277579a09c346/8005523754887120771" poster="https://f48-zpg-r.zdn.vn/4278026175757984954/5eda93de28769428cd67.jpg" controls="" controlslist="nofullscreen nodownload noremoteplayback noplaybackrate" disablepictureinpicture="" style="width: 100%;">...</video>
  (Session info: chrome=132.0.6834.83)
Stacktrace:
	GetHandleVerifier [0x00007FF66407FBE5+28741]
	(No symbol) [0x00007FF663FE7890]
	(No symbol) [0x00007FF663E84FDA]
	(No symbol) [0x00007FF663EE1048]
	(No symbol) [0x00007FF663EDEA92]
	(No symbol) [0x00007FF663EDBBA1]
	(No symbol) [0x00007FF663EDAB01]
	(No symbol

In [102]:
a = driver.find_elements(By.XPATH,"//div[@class='clickable file-message__actions download']/div")
len(a)

0